In [31]:
import cv2
import math
import os

# CREATE TILES

[0,0]: upper left corner 

img_shape: [height, width]


In [32]:
dirname = './images/train2018'
dirLabel = './labels/train2018'
files = [f[:-4] for f in os.listdir(dirname) if f[-4:].lower() == '.jpg'] # getting a list of all imageNames in folder


In [33]:
def getLabel(filepath, i, j ):
    with open(filepath) as fp:  
        line = fp.readline()
        
        cnt = 1
        while line:
            ord = line.split()
            #print('ANNOTATION: %s'%(line))
            isInLine(ord, i, j)
            line = fp.readline()
            cnt += 1

### INTERSECTION
<img src="img/image.png" style="height: 250px; float: left"/>
#### INPUT: 
rec1: bottom-left(0, 0), top-right(10, 8),

rec2: bottom-left(2, 3), top-right(7, 9)
#### OUTPUT: 
(2, 3) (7, 8) (2, 8) (7, 3)

https://www.geeksforgeeks.org/intersecting-rectangle-when-bottom-left-and-top-right-corners-of-two-rectangles-are-given/ 


In [51]:
def FindPoints(x1, y1, x2, y2, x3, y3, x4, y4, imgName): 
    
    # gives bottom-left point 
    # of intersection rectangle 
    x5 = max(x1, x3) 
    y5 = max(y1, y3) 
  
    # gives top-right point 
    # of intersection rectangle 
    x6 = min(x2, x4) 
    y6 = min(y2, y4) 
  
    # no intersection 
    if (x5 > x6 or y5 > y6) : 
        #print("NO INTERSECTION")
        return
        
    #New annotation
    width = max(x5,x6)-min(x5,x6)
    height = max(y5,y6)-min(y5,y6)
    x_center = min(x5,x6) + 0.5*width
    y_center = min(y5,y6) + 0.5*height
    new_annotation = 'Annotation: %s %s %s %s'%(x_center, y_center, width, height)
    print(new_annotation)
    
    

<b>Annotation format</b>: class x_center y_center width height 

Box coordinates are normalized xywh format (from 0 - 1).

<b>j</b>: 


In [67]:
def isInLine(annotation, i, j):
    #Annotation bbox: 
    print(annotation)
    aclas, axcenter, aycenter, awidth, aheight = annotation
    axmin, axmax, aymin, aymax = float(axcenter)-(0.5*float(awidth)), float(axcenter)+(0.5*float(awidth)), float(aycenter)-(0.5*float(aheight)), float(aycenter)+(0.5*float(aheight))
    
    #Tile bbox
    tw = 0.2  #tilewidth
    txmax, tymax = tw*(i+1), tw*(j+1)
    txmin, tymin = txmax - tw, tymax - tw
    #print('x:[%s,%s], y[%s,%s]'%(txmin,txmax,tymin,tymax))
  
    
    #print('tmn:%s, tmx:%s, tmn:%s, tmx:%s'%(round(txmin,4), round(txmax,4), round(tymin, 5), round(tymax,5)))
    #nr1 = txmin <= axmin <= txmax
    #nr2 = txmin <= axmax <= txmax
    #nr3 = tymin <= aymin <= tymax
    #nr4 = tymin <= aymax <= tymax
    imgName = 'tile_%s_%s'%(str(i), str(j))
    
    intersection = FindPoints(txmin, tymin, txmax, tymax, axmin, aymin, axmax, aymax, imgName)
    

### CREATING TILES
<img src="img/tiles2.png" style="height: 250px; float: left; padding-right: 10px;"/>

Crop image to 5x5 tiles

Original image size: 4000 x 2250

Tile size: 800 x 450

Saving new tiles: "tile_i_j" in folder croppedImages


In [47]:
def cropImg(imgpath, labelpath):
    img = cv2.imread(imgpath) # 4000  ×  2250   600 x 337.5
    
    img_shape = img.shape
    print(img_shape)
    tile_size =  (int(img_shape[1]*0.2), int(img_shape[0]*0.2)) # 800 x 450
    print(tile_size)
    offset = (int(img_shape[1]*0.2), int(img_shape[0]*0.2))

    for i in range(int(math.ceil(img_shape[0]/(offset[1] * 1.0)))):
        for j in range(int(math.ceil(img_shape[1]/(offset[0] * 1.0)))):
            xmin = offset[1]*i
            xmaks = min(offset[1]*i+tile_size[1], img_shape[0])
            ymin = offset[0]*j
            ymaks = min(offset[0]*j+tile_size[0], img_shape[1])
            en = (i+1)*0.2
            to = (j+1)*0.2
            cropped_img = img[xmin:xmaks, ymin:ymaks]
            cv2.imwrite('./croppedImages/tile_%s_%s.png'%(str(i), str(j)), cropped_img)
            print('tile_%s_%s'%(i,j))
            getLabel(labelpath, i, j)
            
            
            

In [48]:
filepath = '%s/%s.txt'%(dirLabel, 'COCO_train2018_dji_0289')
imgpath = '%s/%s.jpg'%(dirname, 'COCO_train2018_dji_0289')

print(imgpath)

./images/train2018/COCO_train2018_dji_0289.jpg


In [49]:
getLabel(filepath,1,1)

NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
NO INTERSECTION
Annotation: 0.21775 0.3846666666666667 0.03049999999999997 0.030666666666666675


In [68]:
cropImg(imgpath, filepath)

(2250, 4000, 3)
(800, 450)
tile_0_0
['2', '0.417375', '0.6111111111111112', '0.01925', '0.02666666666666667']
['2', '0.3615', '0.49822222222222223', '0.0225', '0.032']
['2', '0.34975', '0.44266666666666665', '0.0455', '0.08']
['2', '0.31975', '0.5093333333333333', '0.028', '0.04622222222222222']
['2', '0.1785', '0.8084444444444444', '0.0255', '0.037333333333333336']
['2', '0.557625', '0.47444444444444445', '0.02525', '0.04666666666666667']
['2', '0.5805', '0.444', '0.0205', '0.03111111111111111']
['2', '0.5625', '0.4295555555555556', '0.0195', '0.029777777777777778']
['3', '0.59275', '0.35155555555555557', '0.015', '0.04711111111111111']
['1', '0.81775', '0.32555555555555554', '0.0095', '0.02']
['2', '0.79925', '0.3268888888888889', '0.0265', '0.025333333333333333']
['2', '0.80725', '0.316', '0.0235', '0.03288888888888889']
['2', '0.809', '0.3468888888888889', '0.0325', '0.02']
['2', '0.783', '0.3662222222222222', '0.023', '0.08622222222222223']
['2', '0.63975', '0.8002222222222222', '

tile_3_1
['2', '0.417375', '0.6111111111111112', '0.01925', '0.02666666666666667']
['2', '0.3615', '0.49822222222222223', '0.0225', '0.032']
['2', '0.34975', '0.44266666666666665', '0.0455', '0.08']
['2', '0.31975', '0.5093333333333333', '0.028', '0.04622222222222222']
['2', '0.1785', '0.8084444444444444', '0.0255', '0.037333333333333336']
['2', '0.557625', '0.47444444444444445', '0.02525', '0.04666666666666667']
['2', '0.5805', '0.444', '0.0205', '0.03111111111111111']
['2', '0.5625', '0.4295555555555556', '0.0195', '0.029777777777777778']
['3', '0.59275', '0.35155555555555557', '0.015', '0.04711111111111111']
Annotation: 0.600125 0.35155555555555557 0.00024999999999986144 0.047111111111111104
['1', '0.81775', '0.32555555555555554', '0.0095', '0.02']
['2', '0.79925', '0.3268888888888889', '0.0265', '0.025333333333333333']
Annotation: 0.793 0.3268888888888889 0.014000000000000012 0.02533333333333332
['2', '0.80725', '0.316', '0.0235', '0.03288888888888889']
Annotation: 0.79775 0.316 0.

In [ ]:
tile_size = (800, 450)
offset = (800, 450)
img_shape = [2250, 4000]

In [ ]:
int(math.ceil(img_shape[0]/offset[1]*1.0))

In [ ]:
int(math.ceil(img_shape[1]/(offset[0] * 1.0)))

In [ ]:
i = 0
j = 0

In [ ]:
a = offset[1]*i
b = min(offset[1]*i+tile_size[1], img_shape[0])
c = img_shape[1] - offset[0]*j
d = img_shape[1] - min(offset[0]*j+tile_size[0], img_shape[1])
am, ax = a - 2250, 2250 - a
bm, bx = b - 2250, 2250 - b
cm, cx = c - 4000, 4000 - c
dm, dx = d - 4000, 4000 - d

print('a: %s, ax: %s, am: %s'%(a,ax, am))
print('b: %s, bx: %s, bm: %s'%(b,bx, bm))
print('c: %s, cx: %s, cm: %s'%(c,cx, cm))
print('d: %s, dx: %s, dm: %s'%(d,dx, dm))

In [ ]:
FindPoints(1,1,3,3,2,0,4,2, "hei")

In [ ]:
min(2,3) + 0.5*1

In [ ]:
min(2,3) +0.5

In [ ]:
getLabel(filepath, 0, 0 )

In [61]:
import bisect
def find(num, boundaries):
    if boundaries[0] < num < boundaries[-1]:
        index = bisect.bisect_right(boundaries, num)
        return boundaries[index - 1], boundaries[index]
    return None

#b = range(11)

#CASES = [.75, 0, 0.17, 5, 0.1, 0.9, 0.311, -1, 6.7]

#for n in CASES:
    #print('{} belongs to group {}'.format(n, find(n, r)))

In [9]:
def getTile(filepath):
    with open(filepath) as fp:  
        line = fp.readline()
        
        cnt = 1
        while line:
            ord = line.split()
            #print('ANNOTATION: %s'%(line))
            calculateTile(ord)
            line = fp.readline()
            cnt += 1

In [55]:
def calculateTile(annotation):
    clas, xcenter, ycenter, width, height = annotation
    print(annotation)
    xmin = float(xcenter) - (0.5*float(width))
    xmax = float(xmin) + (float(width))
    ymin = float(ycenter) - (0.5*float(height))
    ymax = float(ymin) + (float(height))
    print('x:[%s,%s], y:[%s,%s]'%(xmin,xmax,ymin,ymax))
    #b = [0,0.2,0.4,0.6,0.8,1]
    #print(find(xmin, c))
    x = getTiles([xmin,xmax])
    y = getTiles([ymin,ymax])
    print(x)
    en = 't_%s_%s'%(y[0],x[0])
    to = 't_%s_%s'%(y[1],x[0])
    tre ='t_%s_%s'%(y[0],x[1])
    fire ='t_%s_%s'%(y[1],x[1])
    print('tiles: %s, %s, %s, %s'%(en,to,tre,fire))
    

In [56]:
calculateTile([0, 0.5, 0.2, 0.1,0.1])

[0, 0.5, 0.2, 0.1, 0.1]
x:[0.45,0.55], y:[0.15000000000000002,0.25]
[2, 2]
tiles: t_0_2, t_1_2, t_0_2, t_1_2


In [ ]:
print(range(11))

In [ ]:
c = np.linspace(0, 1, 6)

In [ ]:
print(c)

In [143]:
def getTiles(coords):
    tiles = []
    for no in coords:
        if(0 <= no < 0.2 ):#and 0 not in tiles):
            tiles.append(0)
        if(0.2 <= no < 0.4 ):#and 1 not in tiles):
            tiles.append(1)
        if(0.4 <= no < 0.6 ):#and 2 not in tiles):
            tiles.append(2)
        if(0.6 <= no < 0.8 ):#and 3 not in tiles):
            tiles.append(3)
        if(0.8 <= no < 1):# and 4 not in tiles):
            tiles.append(4)
    return(range(tiles[0], tiles[1]+1))


In [159]:
def splitAnnotation(annotation):
    clas, xcenter, ycenter, width, height = annotation
    xmin = float(xcenter) - (0.5*float(width))
    xmax = float(xmin) + (float(width))
    ymin = float(ycenter) - (0.5*float(height))
    ymax = float(ymin) + (float(height))
    print('minX:%s, maxX: %s'%(xmin, xmax))
    print('minY:%s, maxY: %s'%(ymin, ymax))
    
    
    xs = getTiles([xmin, xmax])
    ys = getTiles([ymin, ymax])
    
    for x in xs:
        for y in ys:
            #Bbox tile:
            txmin = x*0.2
            txmax = txmin+0.2
            tymin = y*0.2
            tymax = tymin+0.2
            
            #Bbox annotation in tile
            nxmin = txmin
            nxmax = txmax
            nymin = tymin
            nymax = tymax
            
            if(txmin <= xmin <txmax):
                nxmin = xmin
            if(txmin <= xmax <txmax):
                nxmax = xmax
            if(tymin <= ymin <tymax):
                nymin = ymin
            if(tymin <= ymax <tymax):
                nymax = ymax
            #print('Annotation (%s, %s): xmin: %s - xmax: %s - ymin: %s - ymax: %s'%(x,y,nxmin,nxmax,nymin,nymax))
            
            #new annotaion
            xcenter = (nxmax+nxmin)/2
            ycenter = (nymax+nymin)/2
            width = nxmax-nxmin
            height = nymax-nymin
            newA = '%s %s %s %s %s'%(clas, xcenter, ycenter, width, height )
            print(newA)

In [160]:
splitAnnotation(['2', '0.417375', '0.6111111111111112', '0.01925', '0.02666666666666667'])

minX:0.40775, maxX: 0.427
minY:0.5977777777777779, maxY: 0.6244444444444445
2 0.417375 0.598888888888889 0.01924999999999999 0.0022222222222222365
2 0.417375 0.6122222222222222 0.01924999999999999 0.02444444444444438


In [123]:
a =range(1,2)
print(len(a))

1
